# Llama-Chat-7B

In [ ]:
!nvidia-smi

Wed Nov 29 06:30:17 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    23W / 300W |      0MiB / 16384MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Install Packages and Dependencies

In [ ]:
!pip install --quiet torch transformers accelerate bitsandbytes langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 87.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.5/177.5 kB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.0/47.0 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.7 MB/s eta 0:00:00


In [ ]:
from google.colab import drive

drive.mount("/content/drive")
%cd /content/drive/MyDrive/LLM

import os

MAIN_DIR = os.getcwd()
DATA_DIR = os.path.join(MAIN_DIR, "data")
MODEL_DIR = os.path.join(MAIN_DIR, "models")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/LLM


## Import Packages

In [ ]:
import torch
from transformers import AutoTokenizer, LlamaForCausalLM, LlamaTokenizer, BitsAndBytesConfig, GenerationConfig, pipeline

## Load Model

Colab GPU:
- A100: 40 GB
- V100: 16 GB
- T4: 12 GB

For inference:
- 32-bit: 28 GB VRAM => Need A-100
- 16-bit: 14 GB VRAM => Need V-100
- 8-bit: 7 GB => All Colab GPU OK

**BFloat16** & **TF32** Precisions require **Ampere** Architecture and above (A100, A4 etc)

In [ ]:
model_path = os.path.join(MODEL_DIR, "llama-2-hf-7b-chat")

tokenizer = AutoTokenizer.from_pretrained(model_path)
tokenizer.pad_token = tokenizer.eos_token
print(tokenizer.special_tokens_map)

model = LlamaForCausalLM.from_pretrained(
    model_path,
    torch_dtype=torch.float16,
    load_in_8bit=True,
    device_map={'': 0},
)

generation_config = GenerationConfig(
    max_new_tokens=128, temperature=0.2,
    top_k=50, do_sample=True,
)

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer,
                generation_config=generation_config)

{'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '</s>'}


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

## Prompt Construction

In [ ]:
from langchain import PromptTemplate
from langchain.llms import HuggingFacePipeline
from langchain.chains import LLMChain

ModuleNotFoundError: ignored